In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
def mape_score(y_test, pred):
    y_test, pred = np.array(y_test), np.array(pred)
    mape = np.mean(np.abs((y_test - pred) / y_test)) * 100
    return mape

In [3]:
def metrics(test, pred):
    d= {}
    mse = mean_squared_error(test, pred)
    rmse = np.sqrt(mse)
    # r_squared = r2_score(test, pred)
    mape= mape_score(test, pred)

    d['MSE'] = mse
    d['RMSE'] = rmse
    # d['R-squared'] = r_squared
    d['MAPE'] = mape
    
    return d

In [ ]:
import darts
from darts import TimeSeries
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.metrics import mape as mape_darts, r2_score
from darts.datasets import AirPassengersDataset

In [ ]:
path = r'C:\Users\himanshu.verma\vs_code\POC_MGC\Code\nbeats_data_pakshal.csv'
df = pd.read_csv(path)

data_nbeats_new = df.copy()
data_nbeats_new = pd.DataFrame(data_nbeats_new)
data_nbeats_new = data_nbeats_new.reset_index()

# Create a TimeSeries, specifying the time and value columns
series_new = TimeSeries.from_dataframe(data_nbeats_new, 'DATE', 'GROSS_PROFIT_ACT')
series_new

In [ ]:
print(len(series_new))
train_nbeats_new, val_nbeats_new = series_new.split_after(pd.Timestamp("20220930"))
print(len(train_nbeats_new))
print(len(val_nbeats_new))

In [ ]:
model_nbeats_new = NBEATSModel(
    input_chunk_length= 1,
    output_chunk_length=1,
    generic_architecture=True,
    num_stacks=20,
    num_blocks=20,
    num_layers=20,
    layer_widths=20,
    n_epochs=20,
    batch_size=2,
    model_name="nbeats_run",
)

model_nbeats_new.fit(series= train_nbeats_new)

In [ ]:
ts_tpred_new = model_nbeats_new.predict(n= 3,  
	                        # past_covariates=cov_t,
	                        num_samples= 1,   
	                        n_jobs= -1, 
	                        verbose=True)

In [ ]:
param_grid = {
    'n_stacks': [3, 4, 5],
    'stack_widths': [256, 512, 1024],
    'learning_rate': [0.001, 0.01, 0.1],
    # Add other hyperparameters here
}

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=your_nbeats_model,
                           param_grid=param_grid,
                           cv=tscv,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)  # Use parallel processing if available

grid_search.fit(X, y)  # X and y are your time series data and target

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_